Intro to WorkBank dataset


    importance of the dataset to the study

In [9]:
import os
import pandas as pd
import numpy as np
import json
import re
import gensim
import nltk
import requests
import datetime
import seaborn as sns

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora, models
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
from datetime import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
import re

today = datetime.today()

C:\Users\Ghael\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


Import all the necessary libraries, and set the date and time to today.

## Job Portal - WORKBANK

We will be using Beautiful Soup to scrape job posts from WorkBank.com
The following are the relevant data that we will need to take from the website.
- WORKBANK_JOB_TITLE - The title of the Job Post
- WORKBANK_JOB_CATEGORY - The category of the Job Post with relation to the STEM field 
- WORKBANK_JOB_COMPANY - The Company which is looking for applications for the Job Post
- WORKBANK_JOB_DATE - Date and time the Job Post was posted
- WORKBANK_JOB_LOCATION - Location where the Job Post is assigned to
- WORKBANK_JOB_STATUS - This determines the type of the job whether it is full time or not
- WORKBANK_JOB_SALARY - Monthly salary of the job listing in Philippine Pesos (PHP)
- WORKBANK_JOB_EDUCATION - Educational attainment requirements for the applicant
- WORKBANK_JOB_DESCRIPTION - A detailed job description
- WORKBANK_JOB_YEAR_WE - Years of Work Experience required for the job

In [10]:
WORKBANK_JOB_TITLE = []
WORKBANK_JOB_CATEGORY = []
WORKBANK_JOB_COMPANY = []
WORKBANK_JOB_DATE = []
WORKBANK_JOB_LOCATION = []
WORKBANK_JOB_STATUS = []
WORKBANK_JOB_SALARY = []
WORKBANK_JOB_EDUCATION = []
WORKBANK_JOB_DESCRIPTION = []
WORKBANK_JOB_YEAR_WE = []

### CATEGORY - Information Communications Technology

Since we are only looking for Job Posts in the STEM field, we will need to go over the website WorkBank.com and choose only the relevant categories, such as the Information Communications Technology. We will need to scrape every single page in that category result in the following code.

In [11]:
#Information Communications Technology
IT_WORKBANK_URL = 'https://www.workbank.com/job/information-communications-technology-job-openings?wb_q='
IT_WORKBANK = requests.get(IT_WORKBANK_URL)
IT_WORKBANK_soup = BeautifulSoup(IT_WORKBANK.content, 'html.parser')
IT_WORKBANK_PP = IT_WORKBANK_soup.find_all('select',{'class':'wb-pagination-select'})
if (len(IT_WORKBANK_PP)!=0):
    IT_WORKBANK_NUMPAGES=len(IT_WORKBANK_PP[0].select("option"))
else:
    IT_WORKBANK_NUMPAGES=0
IT_WORKBANK_PAGES=[]
if (IT_WORKBANK_NUMPAGES!=0):
    for i in range(1,IT_WORKBANK_NUMPAGES+1):
        IT_WORKBANK_PAGES.append('https://www.workbank.com/job/information-communications-technology-job-openings?page='+ str(i))
else:
    IT_WORKBANK_PAGES.append('https://www.workbank.com/job/information-communications-technology-job-openings?page=1')

Getting the html of the URL of the Information Communications Technology job openings, it can be observed that it contains the list of jobs that we interested in.

In [16]:
souptest = BeautifulSoup(IT_WORKBANK.content, 'html.parser')
souptest

<!DOCTYPE html>
<html lang="en"><head><meta charset="utf-8"/><script async="" src="https://www.googletagmanager.com/gtag/js?id=GTM-TL4H9C5"></script><script> window.dataLayer = window.dataLayer || []; function gtag(){dataLayer.push(arguments);} gtag('js', new Date()); gtag('config', 'UA-120994141-1'); </script><title> Job hiring in Information Communications Technology | Find work @ Workbank </title><meta content="On a job search? You've found the right career website! Find jobs by industry. Sign up for free and find work at Workbank today!" name="description"/><meta content="website" name="og:type"><meta content="Find careers in Information Communications Technology. Sign up now!" name="og:title"><meta content="https://www.workbank.com/job/information-communications-technology-job-openings" name="og:url"/><meta content="See full list of careers in Information Communications Technology. Find work at Workbank today!" name="og:description"/><meta content="summary" name="twitter:card"/><m

In [18]:
i =0
IT_WORKBANK_URLs = IT_WORKBANK_PAGES[i]
IT_WORKBANK_PAGE = requests.get(IT_WORKBANK_URLs)
IT_WORKBANK_PAGE_soup = BeautifulSoup(IT_WORKBANK_PAGE.content, 'html.parser')
IT_WORKBANK_JOBS = IT_WORKBANK_PAGE_soup.find_all('a',{'class':'clearfix'})
IT_WORKBANK_JOB_URLs = re.findall(r'(?s)(?<=href=").*?(?="><h5)',str(IT_WORKBANK_JOBS))
IT_WORKBANK_JOB_DATEs = IT_WORKBANK_PAGE_soup.find_all('p',{'class':'publish-date-card mt-1 text-left mb-0'})
for j in range(len(IT_WORKBANK_JOB_URLs)):
    IT_WORKBANK_JOB_PAGE = requests.get(IT_WORKBANK_JOB_URLs[j])
    IT_WORKBANK_JOB_PAGE_soup = BeautifulSoup(IT_WORKBANK_JOB_PAGE.content, 'html.parser')
    IT_WORKBANK_JOB_PAGE_INFO1 = IT_WORKBANK_JOB_PAGE_soup.find('article',{'class':'job-ad-text-center pl-3'})
    IT_WORKBANK_JOB_TITLE = IT_WORKBANK_JOB_PAGE_INFO1.contents[0].text.strip()
    IT_WORKBANK_JOB_COMPANY = IT_WORKBANK_JOB_PAGE_INFO1.contents[1].text.strip()
    IT_WORKBANK_JOB_SALARY = IT_WORKBANK_JOB_PAGE_INFO1.contents[4].text.strip()
    IT_WORKBANK_JOB_DATEPOSTED = IT_WORKBANK_JOB_DATEs[j].text.strip()
    IT_WORKBANK_JOB_LOCATION = IT_WORKBANK_JOB_PAGE_soup.find('a',{'class':'cls-links'}).text.strip()
    IT_WORKBANK_JOB_INFO2 = IT_WORKBANK_JOB_PAGE_soup.find('ul',{'class':'job-ad-des-ul mb-0'})
    IT_WORKBANK_JOB_STATUS = re.findall(r'(?s)(?<=Job Type</h5><p>).*?(?=</p>)',str(IT_WORKBANK_JOB_INFO2))[0]
    IT_WORKBANK_JOB_EDUCATION = re.findall(r'(?s)(?<=Educational Attainment</h5><p>).*?(?=</p>)',str(IT_WORKBANK_JOB_INFO2))[0]
    IT_WORKBANK_JOB_YEARS_WE = re.findall(r'(?s)(?<=Years of Work Experience</h5><p>).*?(?=</p>)',str(IT_WORKBANK_JOB_INFO2))[0]
    IT_WORKBANK_JOB_CATEGORY = "Information and Communications Technology"
    IT_WORKBANK_JOB_INFO3 = IT_WORKBANK_JOB_PAGE_soup.find('article',{'class':'pl-4 pr-4 pb-0 pt-4'})
    IT_WORKBANK_JOB_DESCRIPTION = IT_WORKBANK_JOB_INFO3.contents[1].getText(separator=u' ')
    WORKBANK_JOB_TITLE.append(IT_WORKBANK_JOB_TITLE)
    WORKBANK_JOB_CATEGORY.append(IT_WORKBANK_JOB_CATEGORY)
    WORKBANK_JOB_COMPANY.append(IT_WORKBANK_JOB_COMPANY)
    WORKBANK_JOB_DATE.append(IT_WORKBANK_JOB_DATEPOSTED)
    WORKBANK_JOB_LOCATION.append(IT_WORKBANK_JOB_LOCATION)
    WORKBANK_JOB_STATUS.append(IT_WORKBANK_JOB_STATUS)
    WORKBANK_JOB_SALARY.append(IT_WORKBANK_JOB_SALARY)
    WORKBANK_JOB_EDUCATION.append(IT_WORKBANK_JOB_EDUCATION)
    WORKBANK_JOB_YEAR_WE.append(IT_WORKBANK_JOB_YEARS_WE)
    WORKBANK_JOB_DESCRIPTION.append(IT_WORKBANK_JOB_DESCRIPTION)

In [ ]:
for i in range(len(IT_WORKBANK_PAGES)):
    IT_WORKBANK_URLs = IT_WORKBANK_PAGES[i]
    IT_WORKBANK_PAGE = requests.get(IT_WORKBANK_URLs)
    IT_WORKBANK_PAGE_soup = BeautifulSoup(IT_WORKBANK_PAGE.content, 'html.parser')
    IT_WORKBANK_JOBS = IT_WORKBANK_PAGE_soup.find_all('a',{'class':'clearfix'})
    IT_WORKBANK_JOB_URLs = re.findall(r'(?s)(?<=href=").*?(?="><h5)',str(IT_WORKBANK_JOBS))
    IT_WORKBANK_JOB_DATEs = IT_WORKBANK_PAGE_soup.find_all('p',{'class':'publish-date-card mt-1 text-left mb-0'})
    for j in range(len(IT_WORKBANK_JOB_URLs)):
        IT_WORKBANK_JOB_PAGE = requests.get(IT_WORKBANK_JOB_URLs[j])
        IT_WORKBANK_JOB_PAGE_soup = BeautifulSoup(IT_WORKBANK_JOB_PAGE.content, 'html.parser')
        IT_WORKBANK_JOB_PAGE_INFO1 = IT_WORKBANK_JOB_PAGE_soup.find('article',{'class':'job-ad-text-center pl-3'})
        IT_WORKBANK_JOB_TITLE = IT_WORKBANK_JOB_PAGE_INFO1.contents[0].text.strip()
        IT_WORKBANK_JOB_COMPANY = IT_WORKBANK_JOB_PAGE_INFO1.contents[1].text.strip()
        IT_WORKBANK_JOB_SALARY = IT_WORKBANK_JOB_PAGE_INFO1.contents[4].text.strip()
        IT_WORKBANK_JOB_DATEPOSTED = IT_WORKBANK_JOB_DATEs[j].text.strip()
        IT_WORKBANK_JOB_LOCATION = IT_WORKBANK_JOB_PAGE_soup.find('a',{'class':'cls-links'}).text.strip()
        IT_WORKBANK_JOB_INFO2 = IT_WORKBANK_JOB_PAGE_soup.find('ul',{'class':'job-ad-des-ul mb-0'})
        IT_WORKBANK_JOB_STATUS = re.findall(r'(?s)(?<=Job Type</h5><p>).*?(?=</p>)',str(IT_WORKBANK_JOB_INFO2))[0]
        IT_WORKBANK_JOB_EDUCATION = re.findall(r'(?s)(?<=Educational Attainment</h5><p>).*?(?=</p>)',str(IT_WORKBANK_JOB_INFO2))[0]
        IT_WORKBANK_JOB_YEARS_WE = re.findall(r'(?s)(?<=Years of Work Experience</h5><p>).*?(?=</p>)',str(IT_WORKBANK_JOB_INFO2))[0]
        IT_WORKBANK_JOB_CATEGORY = "Information and Communications Technology"
        IT_WORKBANK_JOB_INFO3 = IT_WORKBANK_JOB_PAGE_soup.find('article',{'class':'pl-4 pr-4 pb-0 pt-4'})
        IT_WORKBANK_JOB_DESCRIPTION = IT_WORKBANK_JOB_INFO3.contents[1].getText(separator=u' ')
        WORKBANK_JOB_TITLE.append(IT_WORKBANK_JOB_TITLE)
        WORKBANK_JOB_CATEGORY.append(IT_WORKBANK_JOB_CATEGORY)
        WORKBANK_JOB_COMPANY.append(IT_WORKBANK_JOB_COMPANY)
        WORKBANK_JOB_DATE.append(IT_WORKBANK_JOB_DATEPOSTED)
        WORKBANK_JOB_LOCATION.append(IT_WORKBANK_JOB_LOCATION)
        WORKBANK_JOB_STATUS.append(IT_WORKBANK_JOB_STATUS)
        WORKBANK_JOB_SALARY.append(IT_WORKBANK_JOB_SALARY)
        WORKBANK_JOB_EDUCATION.append(IT_WORKBANK_JOB_EDUCATION)
        WORKBANK_JOB_YEAR_WE.append(IT_WORKBANK_JOB_YEARS_WE)
        WORKBANK_JOB_DESCRIPTION.append(IT_WORKBANK_JOB_DESCRIPTION)

In [3]:
#Information Communications Technology
IT_WORKBANK_URL = 'https://www.workbank.com/job/information-communications-technology-job-openings?wb_q='
IT_WORKBANK = requests.get(IT_WORKBANK_URL)
IT_WORKBANK_soup = BeautifulSoup(IT_WORKBANK.content, 'html.parser')
IT_WORKBANK_PP = IT_WORKBANK_soup.find_all('select',{'class':'wb-pagination-select'})
if (len(IT_WORKBANK_PP)!=0):
    IT_WORKBANK_NUMPAGES=len(IT_WORKBANK_PP[0].select("option"))
else:
    IT_WORKBANK_NUMPAGES=0
IT_WORKBANK_PAGES=[]
if (IT_WORKBANK_NUMPAGES!=0): ## Check if there is more than one pages for the category
    for i in range(1,IT_WORKBANK_NUMPAGES+1): ### If there's more than one, append all of the pages.
        IT_WORKBANK_PAGES.append('https://www.workbank.com/job/information-communications-technology-job-openings?page='+ str(i))
else: #### Else just add the first page
    IT_WORKBANK_PAGES.append('https://www.workbank.com/job/information-communications-technology-job-openings?page=1')
for i in range(len(IT_WORKBANK_PAGES)):
    ## Pre-setup for each page to make it easier to categorize and gather the data.
    IT_WORKBANK_URLs = IT_WORKBANK_PAGES[i]
    IT_WORKBANK_PAGE = requests.get(IT_WORKBANK_URLs)
    IT_WORKBANK_PAGE_soup = BeautifulSoup(IT_WORKBANK_PAGE.content, 'html.parser')
    IT_WORKBANK_JOBS = IT_WORKBANK_PAGE_soup.find_all('a',{'class':'clearfix'})
    IT_WORKBANK_JOB_URLs = re.findall(r'(?s)(?<=href=").*?(?="><h5)',str(IT_WORKBANK_JOBS))
    IT_WORKBANK_JOB_DATEs = IT_WORKBANK_PAGE_soup.find_all('p',{'class':'publish-date-card mt-1 text-left mb-0'})
    for j in range(len(IT_WORKBANK_JOB_URLs)):
        ### Data and info gathering.
        IT_WORKBANK_JOB_PAGE = requests.get(IT_WORKBANK_JOB_URLs[j])
        IT_WORKBANK_JOB_PAGE_soup = BeautifulSoup(IT_WORKBANK_JOB_PAGE.content, 'html.parser')
        IT_WORKBANK_JOB_PAGE_INFO1 = IT_WORKBANK_JOB_PAGE_soup.find('article',{'class':'job-ad-text-center pl-3'})
        IT_WORKBANK_JOB_TITLE = IT_WORKBANK_JOB_PAGE_INFO1.contents[0].text.strip()
        IT_WORKBANK_JOB_COMPANY = IT_WORKBANK_JOB_PAGE_INFO1.contents[1].text.strip()
        IT_WORKBANK_JOB_SALARY = IT_WORKBANK_JOB_PAGE_INFO1.contents[4].text.strip()
        IT_WORKBANK_JOB_DATEPOSTED = IT_WORKBANK_JOB_DATEs[j].text.strip()
        IT_WORKBANK_JOB_LOCATION = IT_WORKBANK_JOB_PAGE_soup.find('a',{'class':'cls-links'}).text.strip()
        IT_WORKBANK_JOB_INFO2 = IT_WORKBANK_JOB_PAGE_soup.find('ul',{'class':'job-ad-des-ul mb-0'})
        IT_WORKBANK_JOB_STATUS = re.findall(r'(?s)(?<=Job Type</h5><p>).*?(?=</p>)',str(IT_WORKBANK_JOB_INFO2))[0]
        IT_WORKBANK_JOB_EDUCATION = re.findall(r'(?s)(?<=Educational Attainment</h5><p>).*?(?=</p>)',str(IT_WORKBANK_JOB_INFO2))[0]
        IT_WORKBANK_JOB_YEARS_WE = re.findall(r'(?s)(?<=Years of Work Experience</h5><p>).*?(?=</p>)',str(IT_WORKBANK_JOB_INFO2))[0]
        IT_WORKBANK_JOB_CATEGORY = "Information and Communications Technology"
        IT_WORKBANK_JOB_INFO3 = IT_WORKBANK_JOB_PAGE_soup.find('article',{'class':'pl-4 pr-4 pb-0 pt-4'})
        IT_WORKBANK_JOB_DESCRIPTION = IT_WORKBANK_JOB_INFO3.contents[1].getText(separator=u' ')
        ### Append the gathered data to their respective columns
        WORKBANK_JOB_TITLE.append(IT_WORKBANK_JOB_TITLE)
        WORKBANK_JOB_CATEGORY.append(IT_WORKBANK_JOB_CATEGORY)
        WORKBANK_JOB_COMPANY.append(IT_WORKBANK_JOB_COMPANY)
        WORKBANK_JOB_DATE.append(IT_WORKBANK_JOB_DATEPOSTED)
        WORKBANK_JOB_LOCATION.append(IT_WORKBANK_JOB_LOCATION)
        WORKBANK_JOB_STATUS.append(IT_WORKBANK_JOB_STATUS)
        WORKBANK_JOB_SALARY.append(IT_WORKBANK_JOB_SALARY)
        WORKBANK_JOB_EDUCATION.append(IT_WORKBANK_JOB_EDUCATION)
        WORKBANK_JOB_YEAR_WE.append(IT_WORKBANK_JOB_YEARS_WE)
        WORKBANK_JOB_DESCRIPTION.append(IT_WORKBANK_JOB_DESCRIPTION)

### Category - Construction
The same process is done for the other sources to expand the dataset.

In [ ]:
#Construction
CONSTRUCTION_WORKBANK_URL = 'https://www.workbank.com/job/construction-job-openings?wb_q='
CONSTRUCTION_WORKBANK = requests.get(CONSTRUCTION_WORKBANK_URL)
CONSTRUCTION_WORKBANK_soup = BeautifulSoup(CONSTRUCTION_WORKBANK.content, 'html.parser')
CONSTRUCTION_WORKBANK_PP = CONSTRUCTION_WORKBANK_soup.find_all('div',{'class':'wb-pagination'})
if (len(CONSTRUCTION_WORKBANK_PP)!=0):
    CONSTRUCTION_WORKBANK_NUMPAGES=len(CONSTRUCTION_WORKBANK_PP[0].select("option"))
else:
    CONSTRUCTION_WORKBANK_NUMPAGES=0
CONSTRUCTION_WORKBANK_PAGES=[]
if (CONSTRUCTION_WORKBANK_NUMPAGES!=0): ## Check if there is more than one pages for the category
    for i in range(1,IT_WORKBANK_NUMPAGES+1): ### If there's more than one, append all of the pages.
        CONSTRUCTION_WORKBANK_PAGES.append('https://www.workbank.com/job/construction-job-openings?page='+ str(i))
else: ### Else just add the first page
    CONSTRUCTION_WORKBANK_PAGES.append('https://www.workbank.com/job/construction-job-openings?page=1')
for i in range(len(CONSTRUCTION_WORKBANK_PAGES)):
    ## Pre-setup for each page to make it easier to categorize and gather the data.
    CONSTRUCTION_WORKBANK_URLs = CONSTRUCTION_WORKBANK_PAGES[i]
    CONSTRUCTION_WORKBANK_PAGE = requests.get(CONSTRUCTION_WORKBANK_URLs)
    CONSTRUCTION_WORKBANK_PAGE_soup = BeautifulSoup(CONSTRUCTION_WORKBANK_PAGE.content, 'html.parser')
    CONSTRUCTION_WORKBANK_JOBS = CONSTRUCTION_WORKBANK_PAGE_soup.find_all('a',{'class':'clearfix'})
    CONSTRUCTION_WORKBANK_JOB_URLs = re.findall(r'(?s)(?<=href=").*?(?="><h5)',str(CONSTRUCTION_WORKBANK_JOBS))
    CONSTRUCTION_WORKBANK_JOB_DATEs = CONSTRUCTION_WORKBANK_PAGE_soup.find_all('p',{'class':'publish-date-card mt-1 text-left mb-0'})
    for j in range(len(CONSTRUCTION_WORKBANK_JOB_URLs)):
        ### Data and info gathering.
        CONSTRUCTION_WORKBANK_JOB_PAGE = requests.get(CONSTRUCTION_WORKBANK_JOB_URLs[j])
        CONSTRUCTION_WORKBANK_JOB_PAGE_soup = BeautifulSoup(CONSTRUCTION_WORKBANK_JOB_PAGE.content, 'html.parser')
        CONSTRUCTION_WORKBANK_JOB_PAGE_INFO1 = CONSTRUCTION_WORKBANK_JOB_PAGE_soup.find('article',{'class':'job-ad-text-center pl-3'})
        CONSTRUCTION_WORKBANK_JOB_TITLE = CONSTRUCTION_WORKBANK_JOB_PAGE_INFO1.contents[0].text.strip()
        CONSTRUCTION_WORKBANK_JOB_COMPANY = CONSTRUCTION_WORKBANK_JOB_PAGE_INFO1.contents[1].text.strip()
        CONSTRUCTION_WORKBANK_JOB_SALARY = CONSTRUCTION_WORKBANK_JOB_PAGE_INFO1.contents[4].text.strip()
        CONSTRUCTION_WORKBANK_JOB_DATEPOSTED = CONSTRUCTION_WORKBANK_JOB_DATEs[j].text.strip()
        CONSTRUCTION_WORKBANK_JOB_LOCATION = CONSTRUCTION_WORKBANK_JOB_PAGE_soup.find('a',{'class':'cls-links'}).text.strip()
        CONSTRUCTION_WORKBANK_JOB_INFO2 = CONSTRUCTION_WORKBANK_JOB_PAGE_soup.find('ul',{'class':'job-ad-des-ul mb-0'})
        CONSTRUCTION_WORKBANK_JOB_STATUS = re.findall(r'(?s)(?<=Job Type</h5><p>).*?(?=</p>)',str(CONSTRUCTION_WORKBANK_JOB_INFO2))[0]
        CONSTRUCTION_WORKBANK_JOB_EDUCATION = re.findall(r'(?s)(?<=Educational Attainment</h5><p>).*?(?=</p>)',str(CONSTRUCTION_WORKBANK_JOB_INFO2))[0]
        CONSTRUCTION_WORKBANK_JOB_YEARS_WE = re.findall(r'(?s)(?<=Years of Work Experience</h5><p>).*?(?=</p>)',str(IT_WORKBANK_JOB_INFO2))[0]
        CONSTRUCTION_WORKBANK_JOB_CATEGORY = "Construction"
        CONSTRUCTION_WORKBANK_JOB_INFO3 = CONSTRUCTION_WORKBANK_JOB_PAGE_soup.find('article',{'class':'pl-4 pr-4 pb-0 pt-4'})
        CONSTRUCTION_WORKBANK_JOB_DESCRIPTION = CONSTRUCTION_WORKBANK_JOB_INFO3.contents[1].getText(separator=u' ')
        ### Append the gathered data to their respective columns
        WORKBANK_JOB_TITLE.append(CONSTRUCTION_WORKBANK_JOB_TITLE)
        WORKBANK_JOB_CATEGORY.append(CONSTRUCTION_WORKBANK_JOB_CATEGORY)
        WORKBANK_JOB_COMPANY.append(CONSTRUCTION_WORKBANK_JOB_COMPANY)
        WORKBANK_JOB_DATE.append(CONSTRUCTION_WORKBANK_JOB_DATEPOSTED)
        WORKBANK_JOB_LOCATION.append(CONSTRUCTION_WORKBANK_JOB_LOCATION)
        WORKBANK_JOB_STATUS.append(CONSTRUCTION_WORKBANK_JOB_STATUS)
        WORKBANK_JOB_SALARY.append(CONSTRUCTION_WORKBANK_JOB_SALARY)
        WORKBANK_JOB_EDUCATION.append(CONSTRUCTION_WORKBANK_JOB_EDUCATION)
        WORKBANK_JOB_YEAR_WE.append(CONSTRUCTION_WORKBANK_JOB_YEARS_WE)
        WORKBANK_JOB_DESCRIPTION.append(CONSTRUCTION_WORKBANK_JOB_DESCRIPTION)

### Category - Design and Architecture

In [ ]:
#Design and Architecture
ARCHITECTURE_WORKBANK_URL = 'https://www.workbank.com/job/design-architecture-job-openings?wb_q='
ARCHITECTURE_WORKBANK = requests.get(ARCHITECTURE_WORKBANK_URL)
ARCHITECTURE_WORKBANK_soup = BeautifulSoup(ARCHITECTURE_WORKBANK.content, 'html.parser')
ARCHITECTURE_WORKBANK_PP = ARCHITECTURE_WORKBANK_soup.find_all('div',{'class':'wb-pagination'})
if (len(ARCHITECTURE_WORKBANK_PP)!=0):
    ARCHITECTURE_WORKBANK_NUMPAGES=len(ARCHITECTURE_WORKBANK_PP[0].select("option"))
else:
    ARCHITECTURE_WORKBANK_NUMPAGES=0
ARCHITECTURE_WORKBANK_PAGES=[]
if (ARCHITECTURE_WORKBANK_NUMPAGES!=0): ## Check if there is more than one pages for the category
    for i in range(1,IT_WORKBANK_NUMPAGES+1): ### If there's more than one, append all of the pages.
        ARCHITECTURE_WORKBANK_PAGES.append('https://www.workbank.com/job/design-architecture-job-openings?page='+ str(i))
else: ### Else just add the first page
    ARCHITECTURE_WORKBANK_PAGES.append('https://www.workbank.com/job/design-architecture-job-openings?page=1')
for i in range(len(ARCHITECTURE_WORKBANK_PAGES)):
    ## Pre-setup for each page to make it easier to categorize and gather the data.
    ARCHITECTURE_WORKBANK_URLs = ARCHITECTURE_WORKBANK_PAGES[i]
    ARCHITECTURE_WORKBANK_PAGE = requests.get(ARCHITECTURE_WORKBANK_URLs)
    ARCHITECTURE_WORKBANK_PAGE_soup = BeautifulSoup(ARCHITECTURE_WORKBANK_PAGE.content, 'html.parser')
    ARCHITECTURE_WORKBANK_JOBS = ARCHITECTURE_WORKBANK_PAGE_soup.find_all('a',{'class':'clearfix'})
    ARCHITECTURE_WORKBANK_JOB_URLs = re.findall(r'(?s)(?<=href=").*?(?="><h5)',str(ARCHITECTURE_WORKBANK_JOBS))
    ARCHITECTURE_WORKBANK_JOB_DATEs = ARCHITECTURE_WORKBANK_PAGE_soup.find_all('p',{'class':'publish-date-card mt-1 text-left mb-0'})
    for j in range(len(ARCHITECTURE_WORKBANK_JOB_URLs)):
        ### Data and info gathering.
        ARCHITECTURE_WORKBANK_JOB_PAGE = requests.get(ARCHITECTURE_WORKBANK_JOB_URLs[j])
        ARCHITECTURE_WORKBANK_JOB_PAGE_soup = BeautifulSoup(ARCHITECTURE_WORKBANK_JOB_PAGE.content, 'html.parser')
        ARCHITECTURE_WORKBANK_JOB_PAGE_INFO1 = ARCHITECTURE_WORKBANK_JOB_PAGE_soup.find('article',{'class':'job-ad-text-center pl-3'})
        ARCHITECTURE_WORKBANK_JOB_TITLE = ARCHITECTURE_WORKBANK_JOB_PAGE_INFO1.contents[0].text.strip()
        ARCHITECTURE_WORKBANK_JOB_COMPANY = ARCHITECTURE_WORKBANK_JOB_PAGE_INFO1.contents[1].text.strip()
        ARCHITECTURE_WORKBANK_JOB_SALARY = ARCHITECTURE_WORKBANK_JOB_PAGE_INFO1.contents[4].text.strip()
        ARCHITECTURE_WORKBANK_JOB_DATEPOSTED = ARCHITECTURE_WORKBANK_JOB_DATEs[j].text.strip()
        ARCHITECTURE_WORKBANK_JOB_LOCATION = ARCHITECTURE_WORKBANK_JOB_PAGE_soup.find('a',{'class':'cls-links'}).text.strip()
        ARCHITECTURE_WORKBANK_JOB_INFO2 = ARCHITECTURE_WORKBANK_JOB_PAGE_soup.find('ul',{'class':'job-ad-des-ul mb-0'})
        ARCHITECTURE_WORKBANK_JOB_STATUS = re.findall(r'(?s)(?<=Job Type</h5><p>).*?(?=</p>)',str(ARCHITECTURE_WORKBANK_JOB_INFO2))[0]
        ARCHITECTURE_WORKBANK_JOB_EDUCATION = re.findall(r'(?s)(?<=Educational Attainment</h5><p>).*?(?=</p>)',str(ARCHITECTURE_WORKBANK_JOB_INFO2))[0]
        ARCHITECTURE_WORKBANK_JOB_YEARS_WE = re.findall(r'(?s)(?<=Years of Work Experience</h5><p>).*?(?=</p>)',str(IT_WORKBANK_JOB_INFO2))[0]
        ARCHITECTURE_WORKBANK_JOB_CATEGORY = "Design and Architecture"
        ARCHITECTURE_WORKBANK_JOB_INFO3 = ARCHITECTURE_WORKBANK_JOB_PAGE_soup.find('article',{'class':'pl-4 pr-4 pb-0 pt-4'})
        ARCHITECTURE_WORKBANK_JOB_DESCRIPTION = ARCHITECTURE_WORKBANK_JOB_INFO3.contents[1].getText(separator=u' ')
        ### Append the gathered data to their respective columns
        WORKBANK_JOB_TITLE.append(ARCHITECTURE_WORKBANK_JOB_TITLE)
        WORKBANK_JOB_CATEGORY.append(ARCHITECTURE_WORKBANK_JOB_CATEGORY)
        WORKBANK_JOB_COMPANY.append(ARCHITECTURE_WORKBANK_JOB_COMPANY)
        WORKBANK_JOB_DATE.append(ARCHITECTURE_WORKBANK_JOB_DATEPOSTED)
        WORKBANK_JOB_LOCATION.append(ARCHITECTURE_WORKBANK_JOB_LOCATION)
        WORKBANK_JOB_STATUS.append(ARCHITECTURE_WORKBANK_JOB_STATUS)
        WORKBANK_JOB_SALARY.append(ARCHITECTURE_WORKBANK_JOB_SALARY)
        WORKBANK_JOB_EDUCATION.append(ARCHITECTURE_WORKBANK_JOB_EDUCATION)
        WORKBANK_JOB_YEAR_WE.append(ARCHITECTURE_WORKBANK_JOB_YEARS_WE)
        WORKBANK_JOB_DESCRIPTION.append(ARCHITECTURE_WORKBANK_JOB_DESCRIPTION)

### Category - Agriculture and Wildlife Conservation

In [ ]:
#Agriculture and Wildlife Conservation
AGRICULTURE_WORKBANK_URL = 'https://www.workbank.com/job/agriculture-wildlife-conservation-job-openings?wb_q='
AGRICULTURE_WORKBANK = requests.get(AGRICULTURE_WORKBANK_URL)
AGRICULTURE_WORKBANK_soup = BeautifulSoup(AGRICULTURE_WORKBANK.content, 'html.parser')
AGRICULTURE_WORKBANK_PP = AGRICULTURE_WORKBANK_soup.find_all('div',{'class':'wb-pagination'})
if (len(AGRICULTURE_WORKBANK_PP)!=0):
    AGRICULTURE_WORKBANK_NUMPAGES=len(AGRICULTURE_WORKBANK_PP[0].select("option"))
else:
    AGRICULTURE_WORKBANK_NUMPAGES=0
AGRICULTURE_WORKBANK_PAGES=[]
if (AGRICULTURE_WORKBANK_NUMPAGES!=0): ## Check if there is more than one pages for the category
    for i in range(1,IT_WORKBANK_NUMPAGES+1): ### If there's more than one, append all of the pages.
        AGRICULTURE_WORKBANK_PAGES.append('https://www.workbank.com/job/agriculture-wildlife-conservation-job-openings?page='+ str(i))
else: ### Else just add the first page
    AGRICULTURE_WORKBANK_PAGES.append('https://www.workbank.com/job/agriculture-wildlife-conservation-job-openings?page=1')
for i in range(len(AGRICULTURE_WORKBANK_PAGES)):
    ## Pre-setup for each page to make it easier to categorize and gather the data.
    AGRICULTURE_WORKBANK_URLs = AGRICULTURE_WORKBANK_PAGES[i]
    AGRICULTURE_WORKBANK_PAGE = requests.get(AGRICULTURE_WORKBANK_URLs)
    AGRICULTURE_WORKBANK_PAGE_soup = BeautifulSoup(AGRICULTURE_WORKBANK_PAGE.content, 'html.parser')
    AGRICULTURE_WORKBANK_JOBS = AGRICULTURE_WORKBANK_PAGE_soup.find_all('a',{'class':'clearfix'})
    AGRICULTURE_WORKBANK_JOB_URLs = re.findall(r'(?s)(?<=href=").*?(?="><h5)',str(AGRICULTURE_WORKBANK_JOBS))
    AGRICULTURE_WORKBANK_JOB_DATEs = AGRICULTURE_WORKBANK_PAGE_soup.find_all('p',{'class':'publish-date-card mt-1 text-left mb-0'})
    for j in range(len(AGRICULTURE_WORKBANK_JOB_URLs)):
        ### Data and info gathering.
        AGRICULTURE_WORKBANK_JOB_PAGE = requests.get(AGRICULTURE_WORKBANK_JOB_URLs[j])
        AGRICULTURE_WORKBANK_JOB_PAGE_soup = BeautifulSoup(AGRICULTURE_WORKBANK_JOB_PAGE.content, 'html.parser')
        AGRICULTURE_WORKBANK_JOB_PAGE_INFO1 = AGRICULTURE_WORKBANK_JOB_PAGE_soup.find('article',{'class':'job-ad-text-center pl-3'})
        AGRICULTURE_WORKBANK_JOB_TITLE = AGRICULTURE_WORKBANK_JOB_PAGE_INFO1.contents[0].text.strip()
        AGRICULTURE_WORKBANK_JOB_COMPANY = AGRICULTURE_WORKBANK_JOB_PAGE_INFO1.contents[1].text.strip()
        AGRICULTURE_WORKBANK_JOB_SALARY = AGRICULTURE_WORKBANK_JOB_PAGE_INFO1.contents[4].text.strip()
        AGRICULTURE_WORKBANK_JOB_DATEPOSTED = AGRICULTURE_WORKBANK_JOB_DATEs[j].text.strip()
        AGRICULTURE_WORKBANK_JOB_LOCATION = AGRICULTURE_WORKBANK_JOB_PAGE_soup.find('a',{'class':'cls-links'}).text.strip()
        AGRICULTURE_WORKBANK_JOB_INFO2 = AGRICULTURE_WORKBANK_JOB_PAGE_soup.find('ul',{'class':'job-ad-des-ul mb-0'})
        AGRICULTURE_WORKBANK_JOB_STATUS = re.findall(r'(?s)(?<=Job Type</h5><p>).*?(?=</p>)',str(AGRICULTURE_WORKBANK_JOB_INFO2))[0]
        AGRICULTURE_WORKBANK_JOB_EDUCATION = re.findall(r'(?s)(?<=Educational Attainment</h5><p>).*?(?=</p>)',str(AGRICULTURE_WORKBANK_JOB_INFO2))[0]
        AGRICULTURE_WORKBANK_JOB_YEARS_WE = re.findall(r'(?s)(?<=Years of Work Experience</h5><p>).*?(?=</p>)',str(IT_WORKBANK_JOB_INFO2))[0]
        AGRICULTURE_WORKBANK_JOB_CATEGORY = "Agriculture and Wildlife Conservation"
        AGRICULTURE_WORKBANK_JOB_INFO3 = AGRICULTURE_WORKBANK_JOB_PAGE_soup.find('article',{'class':'pl-4 pr-4 pb-0 pt-4'})
        AGRICULTURE_WORKBANK_JOB_DESCRIPTION = AGRICULTURE_WORKBANK_JOB_INFO3.contents[1].getText(separator=u' ')
        ### Append the gathered data to their respective columns
        WORKBANK_JOB_TITLE.append(AGRICULTURE_WORKBANK_JOB_TITLE)
        WORKBANK_JOB_CATEGORY.append(AGRICULTURE_WORKBANK_JOB_CATEGORY)
        WORKBANK_JOB_COMPANY.append(AGRICULTURE_WORKBANK_JOB_COMPANY)
        WORKBANK_JOB_DATE.append(AGRICULTURE_WORKBANK_JOB_DATEPOSTED)
        WORKBANK_JOB_LOCATION.append(AGRICULTURE_WORKBANK_JOB_LOCATION)
        WORKBANK_JOB_STATUS.append(AGRICULTURE_WORKBANK_JOB_STATUS)
        WORKBANK_JOB_SALARY.append(AGRICULTURE_WORKBANK_JOB_SALARY)
        WORKBANK_JOB_YEAR_WE.append(AGRICULTURE_WORKBANK_JOB_YEARS_WE)
        WORKBANK_JOB_EDUCATION.append(AGRICULTURE_WORKBANK_JOB_EDUCATION)
        WORKBANK_JOB_DESCRIPTION.append(AGRICULTURE_WORKBANK_JOB_DESCRIPTION)

### Category - Environmental and Health Safety

In [ ]:
#Environmental and Health Safety
SAFETY_WORKBANK_URL = 'https://www.workbank.com/job/environmental-health-safety-job-openings?wb_q='
SAFETY_WORKBANK = requests.get(SAFETY_WORKBANK_URL)
SAFETY_WORKBANK_soup = BeautifulSoup(SAFETY_WORKBANK.content, 'html.parser')
SAFETY_WORKBANK_PP = SAFETY_WORKBANK_soup.find_all('div',{'class':'wb-pagination'})
if (len(SAFETY_WORKBANK_PP)!=0):
    SAFETY_WORKBANK_NUMPAGES=len(SAFETY_WORKBANK_PP[0].select("option"))
else:
    SAFETY_WORKBANK_NUMPAGES=0
SAFETY_WORKBANK_PAGES=[]
if (SAFETY_WORKBANK_NUMPAGES!=0): ## Check if there is more than one pages for the category
    for i in range(1,IT_WORKBANK_NUMPAGES+1): ### If there's more than one, append all of the pages.
        SAFETY_WORKBANK_PAGES.append('https://www.workbank.com/job/environmental-health-safety-job-openings?page='+ str(i))
else: ### Else just add the first page
    SAFETY_WORKBANK_PAGES.append('https://www.workbank.com/job/environmental-health-safety-job-openings?page=1')
for i in range(len(SAFETY_WORKBANK_PAGES)):
    ## Pre-setup for each page to make it easier to categorize and gather the data.
    SAFETY_WORKBANK_URLs = SAFETY_WORKBANK_PAGES[i]
    SAFETY_WORKBANK_PAGE = requests.get(SAFETY_WORKBANK_URLs)
    SAFETY_WORKBANK_PAGE_soup = BeautifulSoup(SAFETY_WORKBANK_PAGE.content, 'html.parser')
    SAFETY_WORKBANK_JOBS = SAFETY_WORKBANK_PAGE_soup.find_all('a',{'class':'clearfix'})
    SAFETY_WORKBANK_JOB_URLs = re.findall(r'(?s)(?<=href=").*?(?="><h5)',str(SAFETY_WORKBANK_JOBS))
    SAFETY_WORKBANK_JOB_DATEs = SAFETY_WORKBANK_PAGE_soup.find_all('p',{'class':'publish-date-card mt-1 text-left mb-0'})
    for j in range(len(SAFETY_WORKBANK_JOB_URLs)):
        ### Data and info gathering.
        SAFETY_WORKBANK_JOB_PAGE = requests.get(SAFETY_WORKBANK_JOB_URLs[j])
        SAFETY_WORKBANK_JOB_PAGE_soup = BeautifulSoup(SAFETY_WORKBANK_JOB_PAGE.content, 'html.parser')
        SAFETY_WORKBANK_JOB_PAGE_INFO1 = SAFETY_WORKBANK_JOB_PAGE_soup.find('article',{'class':'job-ad-text-center pl-3'})
        SAFETY_WORKBANK_JOB_TITLE = SAFETY_WORKBANK_JOB_PAGE_INFO1.contents[0].text.strip()
        SAFETY_WORKBANK_JOB_COMPANY = SAFETY_WORKBANK_JOB_PAGE_INFO1.contents[1].text.strip()
        SAFETY_WORKBANK_JOB_SALARY = SAFETY_WORKBANK_JOB_PAGE_INFO1.contents[4].text.strip()
        SAFETY_WORKBANK_JOB_DATEPOSTED = SAFETY_WORKBANK_JOB_DATEs[j].text.strip()
        SAFETY_WORKBANK_JOB_LOCATION = SAFETY_WORKBANK_JOB_PAGE_soup.find('a',{'class':'cls-links'}).text.strip()
        SAFETY_WORKBANK_JOB_INFO2 = SAFETY_WORKBANK_JOB_PAGE_soup.find('ul',{'class':'job-ad-des-ul mb-0'})
        SAFETY_WORKBANK_JOB_STATUS = re.findall(r'(?s)(?<=Job Type</h5><p>).*?(?=</p>)',str(SAFETY_WORKBANK_JOB_INFO2))[0]
        SAFETY_WORKBANK_JOB_EDUCATION = re.findall(r'(?s)(?<=Educational Attainment</h5><p>).*?(?=</p>)',str(SAFETY_WORKBANK_JOB_INFO2))[0]
        SAFETY_WORKBANK_JOB_YEARS_WE = re.findall(r'(?s)(?<=Years of Work Experience</h5><p>).*?(?=</p>)',str(IT_WORKBANK_JOB_INFO2))[0]
        SAFETY_WORKBANK_JOB_CATEGORY = "Environmental and Health Safety"
        SAFETY_WORKBANK_JOB_INFO3 = SAFETY_WORKBANK_JOB_PAGE_soup.find('article',{'class':'pl-4 pr-4 pb-0 pt-4'})
        SAFETY_WORKBANK_JOB_DESCRIPTION = SAFETY_WORKBANK_JOB_INFO3.contents[1].getText(separator=u' ')
        ### Append the gathered data to their respective columns
        WORKBANK_JOB_TITLE.append(SAFETY_WORKBANK_JOB_TITLE)
        WORKBANK_JOB_CATEGORY.append(SAFETY_WORKBANK_JOB_CATEGORY)
        WORKBANK_JOB_COMPANY.append(SAFETY_WORKBANK_JOB_COMPANY)
        WORKBANK_JOB_DATE.append(SAFETY_WORKBANK_JOB_DATEPOSTED)
        WORKBANK_JOB_LOCATION.append(SAFETY_WORKBANK_JOB_LOCATION)
        WORKBANK_JOB_STATUS.append(SAFETY_WORKBANK_JOB_STATUS)
        WORKBANK_JOB_SALARY.append(SAFETY_WORKBANK_JOB_SALARY)
        WORKBANK_JOB_YEAR_WE.append(SAFETY_WORKBANK_JOB_YEARS_WE)
        WORKBANK_JOB_EDUCATION.append(SAFETY_WORKBANK_JOB_EDUCATION)
        WORKBANK_JOB_DESCRIPTION.append(SAFETY_WORKBANK_JOB_DESCRIPTION)

### Category - Medical and Healthcare

In [ ]:
#Medical and Healthcare
HEALTH_WORKBANK_URL = 'https://www.workbank.com/job/medical-healthcare-job-openings?wb_q='
HEALTH_WORKBANK = requests.get(HEALTH_WORKBANK_URL)
HEALTH_WORKBANK_soup = BeautifulSoup(HEALTH_WORKBANK.content, 'html.parser')
HEALTH_WORKBANK_PP = HEALTH_WORKBANK_soup.find_all('div',{'class':'wb-pagination'})
if (len(HEALTH_WORKBANK_PP)!=0):
    HEALTH_WORKBANK_NUMPAGES=len(HEALTH_WORKBANK_PP[0].select("option"))
else:
    HEALTH_WORKBANK_NUMPAGES=0
HEALTH_WORKBANK_PAGES=[]
if (HEALTH_WORKBANK_NUMPAGES!=0): ## Check if there is more than one pages for the category
    for i in range(1,IT_WORKBANK_NUMPAGES+1): ### If there's more than one, append all of the pages.
        HEALTH_WORKBANK_PAGES.append('https://www.workbank.com/job/medical-healthcare-job-openings?page='+ str(i))
else: ### Else just add the first page
    HEALTH_WORKBANK_PAGES.append('https://www.workbank.com/job/medical-healthcare-job-openings?page=1')
for i in range(len(HEALTH_WORKBANK_PAGES)):
    ## Pre-setup for each page to make it easier to categorize and gather the data.
    HEALTH_WORKBANK_URLs = HEALTH_WORKBANK_PAGES[i]
    HEALTH_WORKBANK_PAGE = requests.get(HEALTH_WORKBANK_URLs)
    HEALTH_WORKBANK_PAGE_soup = BeautifulSoup(HEALTH_WORKBANK_PAGE.content, 'html.parser')
    HEALTH_WORKBANK_JOBS = HEALTH_WORKBANK_PAGE_soup.find_all('a',{'class':'clearfix'})
    HEALTH_WORKBANK_JOB_URLs = re.findall(r'(?s)(?<=href=").*?(?="><h5)',str(HEALTH_WORKBANK_JOBS))
    HEALTH_WORKBANK_JOB_DATEs = HEALTH_WORKBANK_PAGE_soup.find_all('p',{'class':'publish-date-card mt-1 text-left mb-0'})
    for j in range(len(HEALTH_WORKBANK_JOB_URLs)):
        ### Data and info gathering.
        HEALTH_WORKBANK_JOB_PAGE = requests.get(HEALTH_WORKBANK_JOB_URLs[j])
        HEALTH_WORKBANK_JOB_PAGE_soup = BeautifulSoup(HEALTH_WORKBANK_JOB_PAGE.content, 'html.parser')
        HEALTH_WORKBANK_JOB_PAGE_INFO1 = HEALTH_WORKBANK_JOB_PAGE_soup.find('article',{'class':'job-ad-text-center pl-3'})
        HEALTH_WORKBANK_JOB_TITLE = HEALTH_WORKBANK_JOB_PAGE_INFO1.contents[0].text.strip()
        HEALTH_WORKBANK_JOB_COMPANY = HEALTH_WORKBANK_JOB_PAGE_INFO1.contents[1].text.strip()
        HEALTH_WORKBANK_JOB_SALARY = HEALTH_WORKBANK_JOB_PAGE_INFO1.contents[4].text.strip()
        HEALTH_WORKBANK_JOB_DATEPOSTED = HEALTH_WORKBANK_JOB_DATEs[j].text.strip()
        HEALTH_WORKBANK_JOB_LOCATION = HEALTH_WORKBANK_JOB_PAGE_soup.find('a',{'class':'cls-links'}).text.strip()
        HEALTH_WORKBANK_JOB_INFO2 = HEALTH_WORKBANK_JOB_PAGE_soup.find('ul',{'class':'job-ad-des-ul mb-0'})
        HEALTH_WORKBANK_JOB_STATUS = re.findall(r'(?s)(?<=Job Type</h5><p>).*?(?=</p>)',str(HEALTH_WORKBANK_JOB_INFO2))[0]
        HEALTH_WORKBANK_JOB_EDUCATION = re.findall(r'(?s)(?<=Educational Attainment</h5><p>).*?(?=</p>)',str(HEALTH_WORKBANK_JOB_INFO2))[0]
        HEALTH_WORKBANK_JOB_YEARS_WE = re.findall(r'(?s)(?<=Years of Work Experience</h5><p>).*?(?=</p>)',str(IT_WORKBANK_JOB_INFO2))[0]
        HEALTH_WORKBANK_JOB_CATEGORY = "Medical and Healthcare"
        HEALTH_WORKBANK_JOB_INFO3 = HEALTH_WORKBANK_JOB_PAGE_soup.find('article',{'class':'pl-4 pr-4 pb-0 pt-4'})
        HEALTH_WORKBANK_JOB_DESCRIPTION = HEALTH_WORKBANK_JOB_INFO3.contents[1].getText(separator=u' ')
        ### Append the gathered data to their respective columns
        WORKBANK_JOB_TITLE.append(HEALTH_WORKBANK_JOB_TITLE)
        WORKBANK_JOB_CATEGORY.append(HEALTH_WORKBANK_JOB_CATEGORY)
        WORKBANK_JOB_COMPANY.append(HEALTH_WORKBANK_JOB_COMPANY)
        WORKBANK_JOB_DATE.append(HEALTH_WORKBANK_JOB_DATEPOSTED)
        WORKBANK_JOB_LOCATION.append(HEALTH_WORKBANK_JOB_LOCATION)
        WORKBANK_JOB_STATUS.append(HEALTH_WORKBANK_JOB_STATUS)
        WORKBANK_JOB_SALARY.append(HEALTH_WORKBANK_JOB_SALARY)
        WORKBANK_JOB_YEAR_WE.append(HEALTH_WORKBANK_JOB_YEARS_WE)
        WORKBANK_JOB_EDUCATION.append(HEALTH_WORKBANK_JOB_EDUCATION)
        WORKBANK_JOB_DESCRIPTION.append(HEALTH_WORKBANK_JOB_DESCRIPTION)

### Category - Sciences

In [ ]:
#Sciences
SCIENCES_WORKBANK_URL = 'https://www.workbank.com/job/sciences-job-openings?wb_q='
SCIENCES_WORKBANK = requests.get(SCIENCES_WORKBANK_URL)
SCIENCES_WORKBANK_soup = BeautifulSoup(SCIENCES_WORKBANK.content, 'html.parser')
SCIENCES_WORKBANK_PP = SCIENCES_WORKBANK_soup.find_all('div',{'class':'wb-pagination'})
if (len(SCIENCES_WORKBANK_PP)!=0):
    SCIENCES_WORKBANK_NUMPAGES=len(SCIENCES_WORKBANK_PP[0].select("option"))
else:
    SCIENCES_WORKBANK_NUMPAGES=0
SCIENCES_WORKBANK_PAGES=[]
if (SCIENCES_WORKBANK_NUMPAGES!=0): ## Check if there is more than one pages for the category
    for i in range(1,IT_WORKBANK_NUMPAGES+1): ### If there's more than one, append all of the pages.
        SCIENCES_WORKBANK_PAGES.append('https://www.workbank.com/job/sciences-job-openings?page='+ str(i))
else: ### Else just add the first page
    SCIENCES_WORKBANK_PAGES.append('https://www.workbank.com/job/sciences-job-openings?page=1')
for i in range(len(SCIENCES_WORKBANK_PAGES)):
    ## Pre-setup for each page to make it easier to categorize and gather the data.
    SCIENCES_WORKBANK_URLs = SCIENCES_WORKBANK_PAGES[i]
    SCIENCES_WORKBANK_PAGE = requests.get(SCIENCES_WORKBANK_URLs)
    SCIENCES_WORKBANK_PAGE_soup = BeautifulSoup(SCIENCES_WORKBANK_PAGE.content, 'html.parser')
    SCIENCES_WORKBANK_JOBS = SCIENCES_WORKBANK_PAGE_soup.find_all('a',{'class':'clearfix'})
    SCIENCES_WORKBANK_JOB_URLs = re.findall(r'(?s)(?<=href=").*?(?="><h5)',str(SCIENCES_WORKBANK_JOBS))
    SCIENCES_WORKBANK_JOB_DATEs = SCIENCES_WORKBANK_PAGE_soup.find_all('p',{'class':'publish-date-card mt-1 text-left mb-0'})
    for j in range(len(SCIENCES_WORKBANK_JOB_URLs)):
        ### Data and info gathering.
        SCIENCES_WORKBANK_JOB_PAGE = requests.get(SCIENCES_WORKBANK_JOB_URLs[j])
        SCIENCES_WORKBANK_JOB_PAGE_soup = BeautifulSoup(SCIENCES_WORKBANK_JOB_PAGE.content, 'html.parser')
        SCIENCES_WORKBANK_JOB_PAGE_INFO1 = SCIENCES_WORKBANK_JOB_PAGE_soup.find('article',{'class':'job-ad-text-center pl-3'})
        SCIENCES_WORKBANK_JOB_TITLE = SCIENCES_WORKBANK_JOB_PAGE_INFO1.contents[0].text.strip()
        SCIENCES_WORKBANK_JOB_COMPANY = SCIENCES_WORKBANK_JOB_PAGE_INFO1.contents[1].text.strip()
        SCIENCES_WORKBANK_JOB_SALARY = SCIENCES_WORKBANK_JOB_PAGE_INFO1.contents[4].text.strip()
        SCIENCES_WORKBANK_JOB_DATEPOSTED = SCIENCES_WORKBANK_JOB_DATEs[j].text.strip()
        SCIENCES_WORKBANK_JOB_LOCATION = SCIENCES_WORKBANK_JOB_PAGE_soup.find('a',{'class':'cls-links'}).text.strip()
        SCIENCES_WORKBANK_JOB_INFO2 = SCIENCES_WORKBANK_JOB_PAGE_soup.find('ul',{'class':'job-ad-des-ul mb-0'})
        SCIENCES_WORKBANK_JOB_STATUS = re.findall(r'(?s)(?<=Job Type</h5><p>).*?(?=</p>)',str(SCIENCES_WORKBANK_JOB_INFO2))[0]
        SCIENCES_WORKBANK_JOB_EDUCATION = re.findall(r'(?s)(?<=Educational Attainment</h5><p>).*?(?=</p>)',str(SCIENCES_WORKBANK_JOB_INFO2))[0]
        SCIENCES_WORKBANK_JOB_YEARS_WE = re.findall(r'(?s)(?<=Years of Work Experience</h5><p>).*?(?=</p>)',str(IT_WORKBANK_JOB_INFO2))[0]
        SCIENCES_WORKBANK_JOB_CATEGORY = "Sciences"
        SCIENCES_WORKBANK_JOB_INFO3 = SCIENCES_WORKBANK_JOB_PAGE_soup.find('article',{'class':'pl-4 pr-4 pb-0 pt-4'})
        SCIENCES_WORKBANK_JOB_DESCRIPTION = SCIENCES_WORKBANK_JOB_INFO3.contents[1].getText(separator=u' ')
        ### Append the gathered data to their respective columns
        WORKBANK_JOB_TITLE.append(SCIENCES_WORKBANK_JOB_TITLE)
        WORKBANK_JOB_CATEGORY.append(SCIENCES_WORKBANK_JOB_CATEGORY)
        WORKBANK_JOB_COMPANY.append(SCIENCES_WORKBANK_JOB_COMPANY)
        WORKBANK_JOB_DATE.append(SCIENCES_WORKBANK_JOB_DATEPOSTED)
        WORKBANK_JOB_LOCATION.append(SCIENCES_WORKBANK_JOB_LOCATION)
        WORKBANK_JOB_STATUS.append(SCIENCES_WORKBANK_JOB_STATUS)
        WORKBANK_JOB_SALARY.append(SCIENCES_WORKBANK_JOB_SALARY)
        WORKBANK_JOB_YEAR_WE.append(SCIENCES_WORKBANK_JOB_YEARS_WE)
        WORKBANK_JOB_EDUCATION.append(SCIENCES_WORKBANK_JOB_EDUCATION)
        WORKBANK_JOB_DESCRIPTION.append(SCIENCES_WORKBANK_JOB_DESCRIPTION)

### Category - Actuarial

In [ ]:
#Actuarial
ACTUARIAL_WORKBANK_URL = 'https://www.workbank.com/job/hiring-actuarial'
ACTUARIAL_WORKBANK = requests.get(ACTUARIAL_WORKBANK_URL)
ACTUARIAL_WORKBANK_soup = BeautifulSoup(ACTUARIAL_WORKBANK.content, 'html.parser')
ACTUARIAL_WORKBANK_PP = ACTUARIAL_WORKBANK_soup.find_all('div',{'class':'wb-pagination'})
if (len(ACTUARIAL_WORKBANK_PP)!=0):
    ACTUARIAL_WORKBANK_NUMPAGES=len(ACTUARIAL_WORKBANK_PP[0].select("option"))
else:
    ACTUARIAL_WORKBANK_NUMPAGES=0
ACTUARIAL_WORKBANK_PAGES=[]
if (ACTUARIAL_WORKBANK_NUMPAGES!=0): ## Check if there is more than one pages for the category
    for i in range(1,IT_WORKBANK_NUMPAGES+1): ### If there's more than one, append all of the pages.
        ACTUARIAL_WORKBANK_PAGES.append('https://www.workbank.com/job/hiring-actuarial?page='+ str(i))
else: ### Else just add the first page
    ACTUARIAL_WORKBANK_PAGES.append('https://www.workbank.com/job/hiring-actuarial?page=1')
for i in range(len(ACTUARIAL_WORKBANK_PAGES)):
    ACTUARIAL_WORKBANK_URLs = ACTUARIAL_WORKBANK_PAGES[i]
    ACTUARIAL_WORKBANK_PAGE = requests.get(ACTUARIAL_WORKBANK_URLs)
    ACTUARIAL_WORKBANK_PAGE_soup = BeautifulSoup(ACTUARIAL_WORKBANK_PAGE.content, 'html.parser')
    ACTUARIAL_WORKBANK_JOBS = ACTUARIAL_WORKBANK_PAGE_soup.find_all('a',{'class':'clearfix'})
    ACTUARIAL_WORKBANK_JOB_URLs = re.findall(r'(?s)(?<=href=").*?(?="><h5)',str(ACTUARIAL_WORKBANK_JOBS))
    ACTUARIAL_WORKBANK_JOB_DATEs = ACTUARIAL_WORKBANK_PAGE_soup.find_all('p',{'class':'publish-date-card mt-1 text-left mb-0'})
    for j in range(len(ACTUARIAL_WORKBANK_JOB_URLs)):
        ### Data and info gathering.
        ACTUARIAL_WORKBANK_JOB_PAGE = requests.get(ACTUARIAL_WORKBANK_JOB_URLs[j])
        ACTUARIAL_WORKBANK_JOB_PAGE_soup = BeautifulSoup(ACTUARIAL_WORKBANK_JOB_PAGE.content, 'html.parser')
        ACTUARIAL_WORKBANK_JOB_PAGE_INFO1 = ACTUARIAL_WORKBANK_JOB_PAGE_soup.find('article',{'class':'job-ad-text-center pl-3'})
        ACTUARIAL_WORKBANK_JOB_TITLE = ACTUARIAL_WORKBANK_JOB_PAGE_INFO1.contents[0].text.strip()
        ACTUARIAL_WORKBANK_JOB_COMPANY = ACTUARIAL_WORKBANK_JOB_PAGE_INFO1.contents[1].text.strip()
        ACTUARIAL_WORKBANK_JOB_SALARY = ACTUARIAL_WORKBANK_JOB_PAGE_INFO1.contents[4].text.strip()
        ACTUARIAL_WORKBANK_JOB_DATEPOSTED = ACTUARIAL_WORKBANK_JOB_DATEs[j].text.strip()
        ACTUARIAL_WORKBANK_JOB_LOCATION = ACTUARIAL_WORKBANK_JOB_PAGE_soup.find('a',{'class':'cls-links'}).text.strip()
        ACTUARIAL_WORKBANK_JOB_INFO2 = ACTUARIAL_WORKBANK_JOB_PAGE_soup.find('ul',{'class':'job-ad-des-ul mb-0'})
        ACTUARIAL_WORKBANK_JOB_STATUS = re.findall(r'(?s)(?<=Job Type</h5><p>).*?(?=</p>)',str(ACTUARIAL_WORKBANK_JOB_INFO2))[0]
        ACTUARIAL_WORKBANK_JOB_EDUCATION = re.findall(r'(?s)(?<=Educational Attainment</h5><p>).*?(?=</p>)',str(ACTUARIAL_WORKBANK_JOB_INFO2))[0]
        ACTUARIA_WORKBANK_JOB_YEARS_WE = re.findall(r'(?s)(?<=Years of Work Experience</h5><p>).*?(?=</p>)',str(IT_WORKBANK_JOB_INFO2))[0]
        ACTUARIAL_WORKBANK_JOB_CATEGORY = "Actuarial"
        ACTUARIAL_WORKBANK_JOB_INFO3 = ACTUARIAL_WORKBANK_JOB_PAGE_soup.find('article',{'class':'pl-4 pr-4 pb-0 pt-4'})
        ACTUARIAL_WORKBANK_JOB_DESCRIPTION = ACTUARIAL_WORKBANK_JOB_INFO3.contents[1].getText(separator=u' ')
        ### Append the gathered data to their respective columns
        WORKBANK_JOB_TITLE.append(ACTUARIAL_WORKBANK_JOB_TITLE)
        WORKBANK_JOB_CATEGORY.append(ACTUARIAL_WORKBANK_JOB_CATEGORY)
        WORKBANK_JOB_COMPANY.append(ACTUARIAL_WORKBANK_JOB_COMPANY)
        WORKBANK_JOB_DATE.append(ACTUARIAL_WORKBANK_JOB_DATEPOSTED)
        WORKBANK_JOB_LOCATION.append(ACTUARIAL_WORKBANK_JOB_LOCATION)
        WORKBANK_JOB_STATUS.append(ACTUARIAL_WORKBANK_JOB_STATUS)
        WORKBANK_JOB_SALARY.append(ACTUARIAL_WORKBANK_JOB_SALARY)
        WORKBANK_JOB_YEAR_WE.append(ACTUARIA_WORKBANK_JOB_YEARS_WE)
        WORKBANK_JOB_EDUCATION.append(ACTUARIAL_WORKBANK_JOB_EDUCATION)
        WORKBANK_JOB_DESCRIPTION.append(ACTUARIAL_WORKBANK_JOB_DESCRIPTION)

### Data check
Print the acquired data set to check the gathered data.

In [4]:
workbank={'Website': "Workbank" ,
          'Job Title': WORKBANK_JOB_TITLE, 
          'Category': WORKBANK_JOB_CATEGORY, 
          'Company': WORKBANK_JOB_COMPANY, 
          'Date Posted': WORKBANK_JOB_DATE, 
          'Location': WORKBANK_JOB_LOCATION, 
          'Status': WORKBANK_JOB_STATUS, 
          'Salary': WORKBANK_JOB_SALARY, 
          'Education': WORKBANK_JOB_EDUCATION, 
          'Years of Work Expirience': WORKBANK_JOB_YEAR_WE,
          'Job Description': WORKBANK_JOB_DESCRIPTION}
workbank_df = pd.DataFrame(data=workbank)
workbank_df

,Website,Job Title,Category,Company,Date Posted,Location,Status,Salary,Education,Years of Work Expirience,Job Description
0,Workbank,ServiceNow Administrator,Information and Communications Technology,"Indra Philippines, Inc.",Posted 21 hours ago,Work from Home,Full Time,,Bachelor’s Degree,3 - 5 Years,The ideal candidate will perform day to day su...
1,Workbank,Angular Developer,Information and Communications Technology,"Indra Philippines, Inc.",Posted 1 day ago,Work from Home,Contract,"PHP 80,000 - 90,000",Bachelor’s Degree,3 - 5 Years,- Analyze and understand requirements provided...
2,Workbank,API Tester,Information and Communications Technology,"Indra Philippines, Inc.",Posted 4 days ago,Work from Home,Contract,"PHP 40,000 - 70,000",Bachelor’s Degree,1 - 3 Years,Participate in all Scrum ceremonies (Sprint P...
3,Workbank,QA Tester (Manual/Functional Testing),Information and Communications Technology,"Indra Philippines, Inc.",Posted 4 days ago,Work from Home,Contract,"PHP 40,000 - 75,000",Bachelor’s Degree,3 - 5 Years,Self-organizing and cross functional developme...
4,Workbank,Senior QA Test Engineer,Information and Communications Technology,SnapMart Inc.,Posted 4 days ago,Taguig,Full Time,"PHP 80,000 - 130,000",Bachelor’s Degree,3 - 5 Years,"In this position, you will work closely with o..."
...,...,...,...,...,...,...,...,...,...,...,...
109,Workbank,PH Registered Nurse (PHRN),Medical and Healthcare,Hinduja Global Solutions,Posted 3 months ago,Quezon City,Full Time,"PHP 25,000 - 30,000",Bachelor’s Degree,1 - 3 Years,MINIMUM QUALIFICATIONS: BS NURSING Graduate Mu...
110,Workbank,US Registered Nurse (USRN),Medical and Healthcare,Hinduja Global Solutions,Posted 3 months ago,Quezon City,Full Time,,Bachelor’s Degree,1 - 3 Years,US Registered Nurse (USRN) with 100k Retention...
111,Workbank,Company Nurse,Medical and Healthcare,"Gardenia Bakeries Philippines, Inc.",Posted 3 months ago,Biñan City,Contract,,Bachelor’s Degree,1 - 3 Years,Administer first aid treatments and provide me...
112,Workbank,US Registered Nurse Analyst,Medical and Healthcare,"Accenture, Inc.",Posted 3 months ago,National Capital Region,Full Time,"PHP 35,000 - 40,000",Bachelor’s Degree,1 - 3 Years,Go through a faster and more convenient recrui...


### Storing the Data 
Store the gathered data into a json file

In [5]:
data = workbank_df.to_json(orient='records')
parsed = json.loads(data)
json.dumps(parsed, indent=4) 
with open('workbank.json', 'w') as json_file:
    json.dump(parsed, json_file)

# Data Clean up

### Data Clean up - Salary
Line up the salary values to make the minimum and maximum values uniform.

In [11]:
def salary_seperatorinator(salary):
    if salary:
        str2 = (salary.replace('PHP', ''))
        str3 = (str2.replace(',', ''))
        stroutput = [int(s) for s in str3.split() if s.isdigit()]
        if not len(stroutput):
            return salary
        else:
            return stroutput[0]
    else:
        return "Unspecified"


In [12]:
def salary_seperatorinator2(salary):
    if salary:
        str2 = (salary.replace('PHP', ''))
        str3 = (str2.replace(',', ''))
        stroutput = [int(s) for s in str3.split() if s.isdigit()]
        if not len(stroutput):
            return salary
        else:
            if len(stroutput) > 1:
                return stroutput[1]
            else:
                return stroutput[0]
    else:
        return "Unspecified"


def date_creatorinator(posted_ago):
    str = posted_ago
    date_checker = str.find("day")
    if date_checker > 0:
        num_posted = [int(s) for s in str.split() if s.isdigit()]
        day = int(today.strftime("%d")) - int(num_posted[0])
        month = today.strftime("%m")
        year =  today.strftime("%y")
        #print (month + "/"  , day , "/" + year)
        return (month + "/"  , day , "/" + year)
    else:
        num_posted = [int(s) for s in str.split() if s.isdigit()]
        month = int(today.strftime("%m")) - int(num_posted[0])
        day = today.strftime("%d")
        year =  today.strftime("%y")
        if month < 1:
            return (month+12 , "/"  , day , "/" + year)
        else:
            return (month , "/"  , day , "/" + year)
        
        
    

### Data Clean up - Date Posted
Since the values of the data in the column of "Date Posted" are too abstract, they will be adjusted to a uniform value with the format of yyyy-mm-dd for clarity.

In [17]:
date_scraped = '5/26/21'
date_scraped = datetime.strptime(date_scraped, '%m/%d/%y')
new_date_posted = []
for index, row in workbank_df.iterrows():
    if "minutes" in row["Date Posted"]: ## Turn "X minutes ago" to a proper date format
        then = date_scraped - timedelta(minutes=int(re.sub('\D', '', row["Date Posted"])))
    elif "hour" in row["Date Posted"]: ## Turn "X hours ago" to a proper date format
        then = date_scraped - timedelta(hours=int(re.sub('\D', '', row["Date Posted"])))
    elif "day" in row["Date Posted"]: ## Turn "X days ago" to a proper date format
        then = date_scraped - timedelta(days=int(re.sub('\D', '', row["Date Posted"])))
    elif "week" in row["Date Posted"]:  ## Turn "X weeks ago" to a proper date format
        then = date_scraped - timedelta(weeks=int(re.sub('\D', '', row["Date Posted"])))
    elif "month" in row["Date Posted"]:  ## Turn "X months ago" to a proper date format
        then = date_scraped - relativedelta(months=int(re.sub('\D', '', row["Date Posted"])))
    elif "year" in row["Date Posted"]:  ## Turn "X years ago" to a proper date format
        then = date_scraped - relativedelta(years=int(re.sub('\D', '', row["Date Posted"])))
    new_date_posted.append(then.date()) ## append the new date to a temporary row
workbank_df["Date Posted"] = new_date_posted ## update all rows with the temporary data.

In [18]:
##workbank_df["Date"]= workbank_df["Date Posted"].apply(date_creatorinator)
workbank_df

,Job Title,Category,Company,Date Posted,Location,Status,Salary,Education,Job Description
0,API Tester,Information and Communications Technology,"Indra Philippines, Inc.",2021-05-05,Work from Home,Contract,"PHP 40,000 - 70,000",Bachelor’s Degree,Participate in all Scrum ceremonies (Sprint P...
1,QA Tester (Manual/Functional Testing),Information and Communications Technology,"Indra Philippines, Inc.",2021-05-05,Work from Home,Contract,"PHP 40,000 - 75,000",Bachelor’s Degree,Self-organizing and cross functional developme...
2,Senior QA Test Engineer,Information and Communications Technology,SnapMart Inc.,2021-05-05,Taguig,Full Time,"PHP 80,000 - 130,000",Bachelor’s Degree,"In this position, you will work closely with o..."
3,Ecommerce Product Manager,Information and Communications Technology,SnapMart Inc.,2021-05-05,Work from Home,Full Time,"PHP 80,000 - 130,000",Bachelor’s Degree,"JOB SUMMARY: As a Product Manager at Snapmart,..."
4,BackEnd Developer,Information and Communications Technology,SnapMart Inc.,2021-05-05,Taguig,Full Time,"PHP 80,000 - 130,000",Bachelor’s Degree,"As a Senior Magento developer, you’ll be resp..."
...,...,...,...,...,...,...,...,...,...
98,PH Registered Nurse (PHRN),Medical and Healthcare,Hinduja Global Solutions,2021-02-07,Quezon City,Full Time,"PHP 25,000 - 30,000",Bachelor’s Degree,MINIMUM QUALIFICATIONS: BS NURSING Graduate Mu...
99,US Registered Nurse (USRN),Medical and Healthcare,Hinduja Global Solutions,2021-02-07,Quezon City,Full Time,,Bachelor’s Degree,US Registered Nurse (USRN) with 100k Retention...
100,Company Nurse,Medical and Healthcare,"Gardenia Bakeries Philippines, Inc.",2021-02-07,Biñan City,Contract,,Bachelor’s Degree,Administer first aid treatments and provide me...
101,US Registered Nurse Analyst,Medical and Healthcare,"Accenture, Inc.",2021-02-07,National Capital Region,Full Time,"PHP 35,000 - 40,000",Bachelor’s Degree,Go through a faster and more convenient recrui...


# Pinoy Jobs

In [15]:
PinoyJobs_URL_List = []
PinoyJobs_URL_List.append("https://pinoyjobs.ph/job-hiring/category/engineering-construction-electrical/")
PinoyJobs_URL_List.append("https://pinoyjobs.ph/job-hiring/category/it-programming-systems-networks/")
PinoyJobs_URL_List.append("https://pinoyjobs.ph/job-hiring/category/manufacturing-production/")
PinoyJobs_URL_List.append("https://pinoyjobs.ph/job-hiring/category/nursing-medical-dental-health/")
PinoyJobs_URL_List.append("https://pinoyjobs.ph/job-hiring/category/sciences-lab-research/")
PinoyJobs_URL_List.append("https://pinoyjobs.ph/job-hiring/category/web-development-design-html-seo/")
job_title_list = []
job_employment_type_list = []
job_jobLocation_list = []
job_dateposted_list = []
job_desc_list = []
job_salary_list = []
job_location_list = []
job_type_list = []
comapny_name_list = []
for web_URL in range(len(PinoyJobs_URL_List)):
    URL = PinoyJobs_URL_List[web_URL]
    print(PinoyJobs_URL_List[web_URL])
    page=requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    for ul in soup.find_all('ul', {"class": "pagination hide-on-small-only"}):
        page_num = ul.find_all("li")
        max_page = int(page_num[len(page_num)-2].text.strip())
        print(page_num[len(page_num)-2].text)
    for web_pages in range(0,max_page): ##max_page
        URL = PinoyJobs_URL_List[web_URL] + "page/{}/".format(web_pages)
        page=requests.get(URL)
        soup = BeautifulSoup(page.content, 'html.parser')
        jobURL_List = []
        for div in soup.find_all('div', class_='card-content'):
             for a in div.find_all('a', href=True):
                jobURL_List.append(a['href'])
                ##print ("Found the URL:", a['href'])
        for k in range(len(jobURL_List)): ##len(jobURL_List)
            jobURL = jobURL_List[k]
            ##print(jobURL)
            jobPage=requests.get(jobURL)
            soupJobs = BeautifulSoup(jobPage.content, 'html.parser')

            job_type = soup.findAll("h1")
            #print(job_type[0].text)
            job_type_list.append(job_type[0].text)

            Job_title = soupJobs.findAll("h1", {"itemprop": "title"})
            #print (titleinfo[0].text)
            job_title_list.append(Job_title[0].text)

            Company_name = soupJobs.find_all('h5',{"itemprop": "hiringOrganization"})
            ##print (Company_name[0].text)
            comapny_name_list.append(Company_name[0].text)

            Job_employmentType = soupJobs.findAll("li", {"itemprop": "employmentType"})
            #print (Job_employmentType[0].text)
            job_employment_type_list.append(Job_employmentType[0].text)

            Job_jobLocation = soupJobs.find_all('li',{"itemprop": "addressLocality"})
            #print (Job_jobLocation[0].text)
            job_jobLocation_list.append(Job_jobLocation[0].text)

            Job_dateposted = soupJobs.find_all('li',{"itemprop": "datePosted"})
            #print (Job_dateposted[0].text)
            job_dateposted_list.append(Job_dateposted[0].text)

            Job_desc =  soupJobs.find_all('div', {"itemprop": "description"})
            #print (Job_desc[0].text)
            job_desc_list.append(Job_desc[0].text)

            Job_salary = soupJobs.find_all('i', {"itemprop": "salary"})
            #print(Job_salary[0].text)
            job_salary_list.append(Job_salary[0].text)

            Job_location = soupJobs.find_all('i')
            #print (Job_location[len(Job_location)-1].text)
            job_location_list.append(Job_location[len(Job_location)-1].text)

    jobs_data={'Website': 'Pinoy Jobs',
               'Job Title': job_title_list, 
               'Category': job_type_list,
               'Company': comapny_name_list,
               'Date Posted': job_dateposted_list,
               'Location': job_location_list,
               'Status': job_employment_type_list, 
               'Salary': job_salary_list,
               #'Office Location': job_jobLocation_list, 
               'Education': "Not Specified / In Description",
               'Years of Work Expirience': "Not Specified / In Description",
               'Job Description': job_desc_list,
               }
    pinoy_jobs_df = pd.DataFrame(data=jobs_data)
    #Website 	Job Title 	Category 	Company 	Date Posted 	Location 	Status 	Salary 	Education 	Job Description

https://pinoyjobs.ph/job-hiring/category/engineering-construction-electrical/
55
https://pinoyjobs.ph/job-hiring/category/it-programming-systems-networks/
59


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


https://pinoyjobs.ph/job-hiring/category/manufacturing-production/
23


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


https://pinoyjobs.ph/job-hiring/category/nursing-medical-dental-health/
27
https://pinoyjobs.ph/job-hiring/category/sciences-lab-research/
5
https://pinoyjobs.ph/job-hiring/category/web-development-design-html-seo/
18


In [14]:
pinoy_jobs_df

,Website,Job Title,Category,Company,Date Posted,Location,Status,Salary,Education,Job Description
0,Pinoy Jobs,Project Manager,"Jobs in Engineering, Construction & Electrical",C.M Pancho Construction Inc.,"Posted on January 17, 2020","The Forum Bldg., #71-A Scout Borromeo St. Brgy...",Full-time,"₱40,000 - ₱80,000",Not Specified / In Description,Description: Responsible for the performance a...
1,Pinoy Jobs,Salesforce Commerce Cloud Front-End Developer,"Jobs in IT, Programming, Systems & Networks",HCM Nexus,"Posted on November 19, 2020",Homebased,Full-time,₱60000 - ₱120000,Not Specified / In Description,Description: • Take ownership over big compone...
2,Pinoy Jobs,"Junior Paint Chemist – Cabuyao, Laguna","Jobs in Manufacturing, Production","Super Globe, Inc.","Posted on May 30, 2020","Cabuyao, Laguna",Full-time,upon interview,Not Specified / In Description,Work : Monday – SaturdayWork time: 7:30am – 4:...
3,Pinoy Jobs,Therapist (Registered Nurse),"Jobs in Nursing, Medical, Dental & Health",KimRey Face and Body Contouring Lounge Inc.,"Posted on August 20, 2019",Units 8 & 9 3/F Richmond Centre 8011 E. Rodrig...,Full-time,₱15000 - ₱20000,Not Specified / In Description,Description: The Nurse performs variety of fac...
4,Pinoy Jobs,Chemist,"Jobs in Sciences, Lab, R&D",Metro Cleanwaste Corporation,"Posted on August 22, 2019","103 Progress Avenue, P1 GIZ, Carmelray Industr...",Full-time,₱10000 - ₱15000,Not Specified / In Description,Description: – Performs physical and chemical ...
5,Pinoy Jobs,Web Developer,"Jobs in Web Development & Design, HTML, SEO",getFIFO,"Posted on April 17, 2019",Online,Full-time,Negotiable,Not Specified / In Description,getFIFO.io is looking for a new FULL TIME Back...


In [ ]:
new_date_posted = []
for index, row in pinoy_jobs_df.iterrows():
    new_date_posted.append(datetime.strptime(row["Date Posted"], 'Posted on %B %d, %Y'))
pinoy_jobs_df["Date Posted"] = new_date_posted

# Cleaning

In [20]:
def salary_seperatorinator(salary):
    ##str = "₱40,000 - ₱80,000"
    str2 = (salary.replace('₱', ''))
    str3 = (str2.replace(',', ''))
    stroutput = [int(s) for s in str3.split() if s.isdigit()]
    if not len(stroutput):
        return -1
    else:
        return stroutput[0]


In [21]:
def salary_seperatorinator2(salary):
    ##str = "₱40,000 - ₱80,000"
    str2 = (salary.replace('₱', ''))
    str3 = (str2.replace(',', ''))
    stroutput = [int(s) for s in str3.split() if s.isdigit()]
    if not len(stroutput):
        return -1
    else:
        return stroutput[1]

def date_changerinator(date):
    str2 = (date.replace('Posted on ', ''))
    str3 = (str2.replace(',', ''))
    datetime_object = datetime.strptime(str3, "%B %d %Y")
    new_format_1 = datetime_object.strftime('%m/%d/%y')
    return (new_format_1)

In [23]:
pinoy_jobs_df

,Job Type,Job Title,Company,Status,Office Location,Date Posted,Job Description,Salary,Location
0,"Jobs in Engineering, Construction & Electrical",Project Manager,C.M Pancho Construction Inc.,Full-time,Quezon City,2020-01-17,Description: Responsible for the performance a...,"₱40,000 - ₱80,000","The Forum Bldg., #71-A Scout Borromeo St. Brgy..."
1,"Jobs in IT, Programming, Systems & Networks",Salesforce Commerce Cloud Front-End Developer,HCM Nexus,Full-time,Makati,2020-11-19,Description: • Take ownership over big compone...,₱60000 - ₱120000,Homebased
2,"Jobs in Manufacturing, Production","Junior Paint Chemist – Cabuyao, Laguna","Super Globe, Inc.",Full-time,Cabuyao,2020-05-30,Work : Monday – SaturdayWork time: 7:30am – 4:...,upon interview,"Cabuyao, Laguna"
3,"Jobs in Nursing, Medical, Dental & Health",Therapist (Registered Nurse),KimRey Face and Body Contouring Lounge Inc.,Full-time,Quezon City,2019-08-20,Description: The Nurse performs variety of fac...,₱15000 - ₱20000,Units 8 & 9 3/F Richmond Centre 8011 E. Rodrig...
4,"Jobs in Sciences, Lab, R&D",Chemist,Metro Cleanwaste Corporation,Full-time,Calamba,2019-08-22,Description: – Performs physical and chemical ...,₱10000 - ₱15000,"103 Progress Avenue, P1 GIZ, Carmelray Industr..."
5,"Jobs in Web Development & Design, HTML, SEO",Web Developer,getFIFO,Full-time,Manila,2019-04-17,getFIFO.io is looking for a new FULL TIME Back...,Negotiable,Online


pinoy_jobs_df["Min Salary"]= pinoy_jobs_df["Salary"].apply(salary_seperatorinator)
pinoy_jobs_df["Max Salary"]= pinoy_jobs_df["Salary"].apply(salary_seperatorinator2)
pinoy_jobs_df["Date Posted"] = pinoy_jobs_df["Date Posted"].apply(date_changerinator)
pinoy_jobs_df

In [ ]:
test_plot = sns.catplot(x="Min Salary",data=pinoy_jobs_df["Min Salary" > 0])

In [ ]:
pinoy_jobs_df["Min Salary"] = pd.to_numeric(pinoy_jobs_df["Min Salary"], downcast="float")

# Monster Jobs

In [6]:
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) '\
           'AppleWebKit/537.36 (KHTML, like Gecko) '\
           'Chrome/75.0.3770.80 Safari/537.36'}
URL = "https://www.monster.com.ph/search/it-computers-software-jobs-1?"
page=requests.get(URL,headers=headers)
soup = BeautifulSoup(page.content, 'html.parser')